In [87]:
import numpy as np
import random

In [2]:
with open('genome.txt', 'w') as gfile:
    gfile.write('test')

In [331]:
def write_genome(gene_groups, fname = 'genome.txt'):
    with open(fname, 'w') as gfile:
        for i in range(len(gene_groups)):
            gfile.write('>g' + str(i) + '\n')
            for g in gene_groups[i]:
                if g[0] == g[-1] and len(g) >= 2:
                    gfile.write('C: ' + ' '.join([str(x) for x in g[:-1]]) + '\n')
                else:
                    gfile.write('L: ' + ' '.join([str(x) for x in g]) + '\n')
            gfile.write('\n')

In [36]:
write_genome([[[1,2,3,-4,-5,6], [7,8,-9,-10]], [[1,2,3,5,4,6], [7,8,-9,-10,7]]])

In [338]:
def read_genome(fname = 'genome.txt'):
    gene_groups = []
    with open(fname, 'r') as gfile:        
        gene = []
        tag = False
        
        for gline in gfile:
            if gline.startswith('>'):
                if tag and gene:
                    gene_groups.append(gene)
                tag = True
                gene = []
                continue
            
            genes = gline.strip().split()
            if genes == []:
                continue
            if genes[0] == 'C:':
                genes.append(genes[1])
            elif genes[0] != 'L:':
                continue
            gene.append([int(x) for x in genes[1:]])
        if gene:
            gene_groups.append(gene)
    return gene_groups

In [48]:
tmp = read_genome()

In [49]:
tmp

[[[1, 2, 3, -4, -5, 6], [7, 8, -9, -10]],
 [[1, 2, 3, 5, 4, 6], [7, 8, -9, -10, 7]]]

In [137]:
tmp = [x + 1 for x in range(20)]
b1 = 3
b2 = 7

In [138]:
tmp[0: b1] + tmp[b2:]

[1, 2, 3, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]

In [85]:
tmp[b1:b2] + tmp[b1:b1+1]

[4, 5, 6, 7, 4]

In [80]:
[*tmp[0:b1], *[-x for x in reversed(tmp[b1:b2])], *tmp[b2:]]

[1, 2, 3, -7, -6, -5, -4, 8, 9, 10]

In [83]:
tmp[0:b1] + [-x for x in reversed(tmp[b1:b2])] + tmp[b2:]

[1, 2, 3, -7, -6, -5, -4, 8, 9, 10]

In [280]:
def trans_circle(g, p1, p2):
    g = g[:-1]
    p1 %= len(g)
    p2 %= len(g)
    
    if p1 == p2:
        return [g[p1:] + g[:p1]]
    if p2 < p1:
        p1, p2 = p2, p1
        
    if random.random() < 0.5:
        c0 = g[0:p1] + [-x for x in reversed(g[p1:p2])] + g[p2:]
        return [c0 + c0[0:1]]
    else:
        c1 = g[0:p1] + g[p2:]
        c2 = g[p1:p2]
        return [c1 + c1[0:1], c2 + c2[0:1]]

In [281]:
def trans_linear(g, p1, p2):
    size = len(g) + 1
    p1 %= size
    p2 %= size
    
    if p1 == p2:        
        return [g]
    if p2 < p1:
        p1, p2 = p2, p1
        
    if p1 == 0 and p2 == len(g):
        return [g + g[0:1]]
    
    if random.random() < 0.5:
        return [g[0:p1] + [-x for x in reversed(g[p1:p2])] + g[p2:]]
    else:
        return [g[0:p1] + g[p2:], g[p1:p2] + g[p1:p1+1]]

def trans(g, p1, p2):
    if g[0] == g[-1] and len(g) >= 2:
        return trans_circle(g, p1, p2)
    return trans_linear(g, p1, p2)

In [303]:
def trans_cross(g1, g2, p1, p2):
    rand_op = random.random()
    if g1[0] == g1[-1] and g2[0] == g2[-1] and len(g1) >= 2 and len(g2) >= 2:
        # circle and circle
        if rand_op < 0.5:
            res = g1[:p1] + g2[p2:-1] + g2[:p2] + g1[p1:-1]          
        else:
            res = g1[:p1] + [-x for x in reversed(g2[:p2])] + \
                    [-x for x in reversed(g2[p2:-1])] + g1[p1:-1]
        return [res + res[0:1]]
        
    if (g1[0] != g1[-1] or len(g1) == 1) and (g2[0] != g2[-1] or len(g2) == 1):
        # linear and linear
        if rand_op < 0.5:
            r1 = g1[:p1] + g2[p2:]
            r2 = g2[:p2] + g1[p1:]
        else:
            r1 = g1[:p1] + [-x for x in reversed(g2[:p2])]
            r2 = [-x for x in reversed(g2[p2:])] + g1[p1:]
        return [x for x in [r1, r2] if x!=[] ]
        
    # linear and circle
    if g1[0] == g1[-1] and len(g1) >= 2:
        c, l = g1, g2
        cp, lp = p1, p2
    else:
        c, l = g2, g1
        cp, lp = p2, p1
    if rand_op < 0.5:
        return [l[:lp] + c[cp:-1] + c[:cp] + l[lp:]]
    else:
        return [l[:lp] + [-x for x in reversed(c[:cp])] + 
                [-x for x in reversed(c[cp:-1])] + l[lp:]]

In [286]:
def trans_op(g, p1, p2):
    size_list = []
    for gene in g:
        size = len(gene)
        if gene[0] == gene[-1] and size >=2 :
            size -= 2
        size_list.append(size)
    p1 %= sum(size_list) + len(size_list)
    p2 %= sum(size_list) + len(size_list)
        
    t1, t2 = 0, 0
    if p1 > p2:
        p2, p1 = p1, p2
    for t1 in range(len(size_list)):
        if p1 <= size_list[t1]:
            break
        p1 -= size_list[t1] + 1
    for t2 in range(len(size_list)):
        if p2 <= size_list[t2]:
            break
        p2 -= size_list[t2] + 1
    # print(t1, p1, t2, p2)
    
    if t1 == t2:
        res = trans(g[t1], p1, p2)
        return g[:t1] + res + g[t1+1:]
    
    res = trans_cross(g[t1], g[t2], p1, p2)
    return g[:t1] + g[t1 + 1:t2] + g[t2 + 1:] + res

In [ ]:
# tmp = [list(range(1, 2001))]
# print(tmp)
# for i in range(5000000):
#     tmp = trans_op(tmp, random.randint(0,3000), random.randint(0,3000))
#     untmp = []
#     for x in tmp:
#         untmp += x
#     if np.unique(untmp).size != 2000:
#         print('error', tmp)
#     if i%50000 == 0:
#         print(i, tmp)
# print(tmp)

In [320]:
genes = []
tmp = [list(range(1, 21))]
genes.append(tmp)
for i in range(50):
    tmp = trans_op(tmp, random.randint(0,3000), random.randint(0,3000))
    genes.append(tmp)

In [321]:
len(genes)

51

In [336]:
for x in genes:
    print(x)

[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]]
[[1, 13, 14, 15, 16, 17, 18, 19, 20], [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 2]]
[[1, 13, 14, 15, 16, 17, 18, -9, -8, -7, -6, -5, -4, -3, -2, -12, -11, -10, 19, 20]]
[[-11, -10, 19, 20], [1, 13, 14, 15, 16, 17, 18, -9, -8, -7, -6, -5, -4, -3, -2, -12, 1]]
[[-11, -10, 19, 14, 15, 16, 17, 18, -9, -8, -7, -6, -5, -4, -3, -2, -12, 1, 13, 20]]
[[-11, -10, 19, 14, 15, 16, 17, 18, -9, -8, -7, -6, -5, -4, -3, -2, -12, 1, 13, 20]]
[[-11, -10, 19, 14, 15, 16, 17, 18, 7, 8, 9, -6, -5, -4, -3, -2, -12, 1, 13, 20]]
[[-11, -3, -2, -12, 1, 13, 20], [-10, 19, 14, 15, 16, 17, 18, 7, 8, 9, -6, -5, -4, -10]]
[[-11, -3, -2, -12, 1, 13, 20], [7, 8, 9, -6, -5, -4, -10, 19, 14, 15, 16, 17, 18]]
[[-11, -3, -4, -10, 19, 14, 15, 16, 17, 18], [7, 8, 9, -6, -5, -2, -12, 1, 13, 20]]
[[-11, -3, -4, -10, 19, 14, 15, 16, 17, 18], [7, 8, 9, 12, 2, 5, 6, 1, 13, 20]]
[[-11, -3, -4, -10, 19, 12, 2, 5, 6, 1, 13, 20], [7, 8, 9, 14, 15, 16, 17, 18]]
[[-

In [339]:
test = read_genome()

In [341]:
for x in test:
    print(x)

[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]]
[[1, 13, 14, 15, 16, 17, 18, 19, 20], [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 2]]
[[1, 13, 14, 15, 16, 17, 18, -9, -8, -7, -6, -5, -4, -3, -2, -12, -11, -10, 19, 20]]
[[-11, -10, 19, 20], [1, 13, 14, 15, 16, 17, 18, -9, -8, -7, -6, -5, -4, -3, -2, -12, 1]]
[[-11, -10, 19, 14, 15, 16, 17, 18, -9, -8, -7, -6, -5, -4, -3, -2, -12, 1, 13, 20]]
[[-11, -10, 19, 14, 15, 16, 17, 18, -9, -8, -7, -6, -5, -4, -3, -2, -12, 1, 13, 20]]
[[-11, -10, 19, 14, 15, 16, 17, 18, 7, 8, 9, -6, -5, -4, -3, -2, -12, 1, 13, 20]]
[[-11, -3, -2, -12, 1, 13, 20], [-10, 19, 14, 15, 16, 17, 18, 7, 8, 9, -6, -5, -4, -10]]
[[-11, -3, -2, -12, 1, 13, 20], [7, 8, 9, -6, -5, -4, -10, 19, 14, 15, 16, 17, 18]]
[[-11, -3, -4, -10, 19, 14, 15, 16, 17, 18], [7, 8, 9, -6, -5, -2, -12, 1, 13, 20]]
[[-11, -3, -4, -10, 19, 14, 15, 16, 17, 18], [7, 8, 9, 12, 2, 5, 6, 1, 13, 20]]
[[-11, -3, -4, -10, 19, 12, 2, 5, 6, 1, 13, 20], [7, 8, 9, 14, 15, 16, 17, 18]]
[[-

In [342]:
genes == test

True

In [343]:
genes[3], test[3]

([[-11, -10, 19, 20],
  [1, 13, 14, 15, 16, 17, 18, -9, -8, -7, -6, -5, -4, -3, -2, -12, 1]],
 [[-11, -10, 19, 20],
  [1, 13, 14, 15, 16, 17, 18, -9, -8, -7, -6, -5, -4, -3, -2, -12, 1]])